# Axioms and Derived Predicates in the Unified Planning Library

In [1]:
import unified_planning as up
import up_symk, up_fast_downward
from unified_planning.shortcuts import *
from unified_planning.io import PDDLWriter, PDDLReader

## A Sokoban task as example

In Sokoban, a character navigates a maze, moving and pushing stones to designated goal squares, with the goal of minimizing stone pushes. The character's intermediate moves between pushes don't count if they allow reaching the stone's adjacent square. In the classical planning formulation, these in-between moves are modeled as actions with zero cost, introducing unnecessary path choices that increase state space and plan length.

Reachability, based on the stone arrangement, is easily expressed as a derived property using axioms. This allows formulating the problem with pushing actions only, enabling the character to "jump" from one stone to another as long as there's a path between them.

The following code is a function that provides the functionality to model the Sokoban planning problem/instance shown below with and without axioms using the Unified Planning Library.

![Image Alt Text](KSokoban-screenshot.png)



In [2]:
sokoban_level = """
#########
###  ####
#     $ #
# #  #$ #
# . .#@ #
#########
""".strip()

def create_sokoban_problem(level=sokoban_level, with_axioms=False):
    problem = up.model.Problem("sokoban")
    
    loc = UserType("location")

    has_player, has_box, adjacent, adjacent_2, can_reach = (
        up.model.Fluent("has_player", BoolType(), l=loc),
        up.model.Fluent("has_box", BoolType(), l=loc),
        up.model.Fluent("adjacent", BoolType(), l1=loc, l2=loc),
        up.model.Fluent("adjacent_2", BoolType(), l1=loc, l2=loc),
        up.model.Fluent("can_reach", DerivedBoolType(), l=loc) if with_axioms else None,
    )

    for fluent in [has_player, has_box, adjacent, adjacent_2, can_reach]:
        if fluent:
            problem.add_fluent(fluent, default_initial_value=False)

    if with_axioms:
        # Axiom 1
        a1 = up.model.Axiom("reach-axiom-1", l=loc)
        a1.set_head(can_reach(a1.parameters[0]))
        a1.add_body_condition(has_player(a1.parameters[0]))
        problem.add_axiom(a1)

        # Axiom 2
        a2 = up.model.Axiom("reach-axiom-2", to=loc)
        to, fr = a2.parameters[0], Variable("from", loc)
        a2.set_head(can_reach(to))
        a2.add_body_condition(Not(has_box(to)))
        a2.add_body_condition(
            Exists(
                And(
                    can_reach(fr),
                    adjacent(fr, to)
                ),
                fr
            )
        )
        problem.add_axiom(a2)

    cost_dict = {}
    
    if not with_axioms:
        # Move action
        move = up.model.InstantaneousAction("move", fr=loc, to=loc)
        fr, to = move.parameters
        move.add_precondition(adjacent(fr, to))
        move.add_precondition(has_player(fr))
        move.add_precondition(Not(has_box(to)))
        move.add_effect(has_player(fr), False)
        move.add_effect(has_player(to), True)
        cost_dict[move] = 0
        problem.add_action(move)

        # Push box
        push_box = up.model.InstantaneousAction("push-box", x=loc, y=loc, z=loc)
        x, y, z = push_box.parameters
        push_box.add_precondition(adjacent(x, y))
        push_box.add_precondition(adjacent(y, z))
        push_box.add_precondition(adjacent_2(x, z))
        push_box.add_precondition(has_player(x))
        push_box.add_precondition(has_box(y))
        push_box.add_precondition(Not(has_box(z)))
        push_box.add_effect(has_player(x), False)
        push_box.add_effect(has_player(y), True)
        push_box.add_effect(has_box(y), False)
        push_box.add_effect(has_box(z), True)
        cost_dict[push_box] = 1
        problem.add_action(push_box)

    if with_axioms:
        # Push box
        push_box = up.model.InstantaneousAction("push-box", l=loc, x=loc, y=loc, z=loc)
        l, x, y, z = push_box.parameters
        push_box.add_precondition(adjacent(x, y))
        push_box.add_precondition(adjacent(y, z))
        push_box.add_precondition(adjacent_2(x, z))
        push_box.add_precondition(has_player(l))
        push_box.add_precondition(can_reach(x))
        push_box.add_precondition(has_box(y))
        push_box.add_precondition(Not(has_box(z)))
        push_box.add_effect(has_player(l), False)
        push_box.add_effect(has_player(y), True)
        push_box.add_effect(has_box(y), False)
        push_box.add_effect(has_box(z), True)
        cost_dict[push_box] = 1
        problem.add_action(push_box)

    problem.add_quality_metric(up.model.metrics.MinimizeActionCosts(cost_dict))

    # Encode level
    level_array = [[char for char in line] for line in level.splitlines()]
    num_col, num_row = len(level_array[0]), len(level_array)
    
    loc_objs = {}

    for x in range(num_col):
        for y in range(num_row):
            entry = level_array[y][x]
            if entry != "#":
                loc_id = f"loc-{x}-{y}"
                cur_obj = up.model.Object(loc_id, loc)
                loc_objs[loc_id] = cur_obj
                problem.add_object(cur_obj)

                if entry == "@":
                    problem.set_initial_value(has_player(cur_obj), True)
                elif entry == "$":
                    problem.set_initial_value(has_box(cur_obj), True)
                elif entry == ".":
                    problem.add_goal(has_box(cur_obj))

    # Set adjacency relationships
    for obj1 in loc_objs:
        for obj2 in loc_objs:
            x1, y1 = map(int, str(obj1).split("-")[1:])
            x2, y2 = map(int, str(obj2).split("-")[1:])
            distance = abs(x1 - x2) + abs(y1 - y2)

            if distance == 1:
                problem.set_initial_value(adjacent(loc_objs[obj1], loc_objs[obj2]), True)
            elif distance == 2 and (x1 == x2 or y1 == y2):
                problem.set_initial_value(adjacent_2(loc_objs[obj1], loc_objs[obj2]), True)

    return problem

## Sokoban without Axioms

Let us now create the Sokoban problem without axioms and solve it with an appropriate classical planner like Fast Downward or Symk.

In [3]:
sokoban_problem = create_sokoban_problem(with_axioms=False)
# print(sokoban_problem)

with OneshotPlanner(problem_kind=sokoban_problem.kind) as planner:
    result = planner.solve(sokoban_problem) # , output_stream=sys.stdout
    if result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
        print(f"{planner.name} found this plan:")
        print(result.plan)
    else:
        print("No plan found.")

NOTE: To disable printing of planning engine credits, add this line to your code: `up.shortcuts.get_environment().credits_stream = None`
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 4 of `/tmp/ipykernel_22559/1694764868.py`, you are using the following planning engine:
  * Engine name: Fast Downward
  * Developers:  Uni Basel team and contributors (cf. https://github.com/aibasel/downward/blob/main/README.md)
  * Description: Fast Downward is a domain-independent classical planning system.

Fast Downward found this plan:
SequentialPlan:
    move(loc-6-4, loc-7-4)
    move(loc-7-4, loc-7-3)
    move(loc-7-3, loc-7-2)
    push-box(loc-7-2, loc-6-2, loc-5-2)
    push-box(loc-6-2, loc-5-2, loc-4-2)
    push-box(loc-5-2, loc-4-2, loc-3-2)
    move(loc-4-2, loc-4-1)
    move(loc-4-1, loc-3-1)
    push-box(loc-3-1, loc-3-2, loc-3-3)
    move(loc-3-2, loc-4-2)
    move(loc-4-2, loc-5-2)
    move(loc-5-2, loc-6-2)
    move(loc-6-2, loc-7-2)
    move(loc-7-2, loc-7-3)
    move

## Sokoban with Axioms

Now, let's create the Sokoban problem with axioms and solve it using a suitable classical planner like Symk. We'll begin by printing the axiom section to demonstrate the internal representation implemented in the Unified Planning Library. It's worth noting that the resulting solution won't include zero-cost player movement actions, as reachability is managed through the background theory established by the axioms.

In [4]:
sokoban_problem = create_sokoban_problem(with_axioms=True)
for axiom in sokoban_problem.axioms:
    print(axiom)

# print(sokoban_problem)

with OneshotPlanner(problem_kind=sokoban_problem.kind) as planner:
    result = planner.solve(sokoban_problem) # , output_stream=sys.stdout
    if result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
        print(f"{planner.name} found this plan:")
        print(result.plan)
    else:
        print("No plan found.")

axiom reach-axiom-1(location l){
   head = can_reach(l) := true
   body = [has_player(l)]
  }
axiom reach-axiom-2(location to){
   head = can_reach(to) := true
   body = [(not has_box(to)), Exists (location from) (can_reach(from) and adjacent(from, to))]
  }
  *** Credits ***
  * In operation mode `OneshotPlanner` at line 7 of `/tmp/ipykernel_22559/1322755656.py`, you are using the following planning engine:
  * Engine name: SymK
  * Developers:  David Speck (cf. https://github.com/speckdavid/symk/blob/master/README.md )
  * Description: SymK is a state-of-the-art domain-independent optimal and top-k planner.

SymK found this plan:
SequentialPlan:
    push-box(loc-6-4, loc-7-2, loc-6-2, loc-5-2)
    push-box(loc-6-2, loc-6-2, loc-5-2, loc-4-2)
    push-box(loc-5-2, loc-5-2, loc-4-2, loc-3-2)
    push-box(loc-4-2, loc-3-1, loc-3-2, loc-3-3)
    push-box(loc-3-2, loc-6-4, loc-6-3, loc-6-2)
    push-box(loc-6-3, loc-7-2, loc-6-2, loc-5-2)
    push-box(loc-6-2, loc-6-2, loc-5-2, loc-4-2)
 

## IO Functionality
To conclude, it's important to highlight that our extended version of the Unified Planning Library also offers comprehensive support for reading and writing PDDL with axioms.

In [5]:
# We do not want to see the credits again and again, so let's disable them from now on.
up.shortcuts.get_environment().credits_stream = None

# Write the problem to the disk
writer = PDDLWriter(sokoban_problem)
writer.write_domain("sokoban-doman.pddl")
writer.write_problem("sokoban-problem.pddl")

# Read the problem again
reader = PDDLReader()
input_problem = reader.parse_problem("sokoban-doman.pddl", "sokoban-problem.pddl")

# Solve it once again
with OneshotPlanner(problem_kind=input_problem.kind) as planner:
    result = planner.solve(input_problem) # , output_stream=sys.stdout
    if result.status in unified_planning.engines.results.POSITIVE_OUTCOMES:
        print(f"{planner.name} found this plan:")
        print(result.plan)
    else:
        print("No plan found.")

SymK found this plan:
SequentialPlan:
    push_box(loc_6_4, loc_7_2, loc_6_2, loc_5_2)
    push_box(loc_6_2, loc_6_2, loc_5_2, loc_4_2)
    push_box(loc_5_2, loc_5_2, loc_4_2, loc_3_2)
    push_box(loc_4_2, loc_3_1, loc_3_2, loc_3_3)
    push_box(loc_3_2, loc_6_4, loc_6_3, loc_6_2)
    push_box(loc_6_3, loc_7_2, loc_6_2, loc_5_2)
    push_box(loc_6_2, loc_6_2, loc_5_2, loc_4_2)
    push_box(loc_5_2, loc_5_2, loc_4_2, loc_3_2)
    push_box(loc_4_2, loc_2_2, loc_3_2, loc_4_2)
    push_box(loc_3_2, loc_3_2, loc_3_3, loc_3_4)
    push_box(loc_3_3, loc_4_4, loc_3_4, loc_2_4)
    push_box(loc_3_4, loc_4_1, loc_4_2, loc_4_3)
    push_box(loc_4_2, loc_4_2, loc_4_3, loc_4_4)
